In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Installing fastapi, uvicorn ,pyngrok , etc into the environment
!pip install fastapi uvicorn torch torchvision pyngrok nest_asyncio

'''
 FastAPI: Defines the API structure and endpoints.
 Uvicorn: Runs the FastAPI application as a server.
 Torch/Torchvision: Handles model loading, preprocessing, and predictions.
 Pyngrok: Creates a public-facing URL to access the FastAPI server.
 Nest-Asyncio: Ensures the FastAPI app can run within the Jupyter notebook.
'''

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00


In [4]:
#installing python-multipart
!pip install python-multipart

'''
python-multipart is a Python library used for parsing multipart/form-data,
a common content type for handling file uploads and form data in HTTP requests.
'''

In [5]:
from fastapi import FastAPI, UploadFile
import torch
import torchvision.transforms as transforms
from PIL import Image
import uvicorn
import nest_asyncio
from pyngrok import ngrok

# Initialize FastAPI app
app = FastAPI()      #calling the FastAPI() method .

# Load the model
model_path = "/content/drive/MyDrive/Spinach_Dataset/resnet50_Spinach_Weights.pth"
model = torch.hub.load("pytorch/vision:v0.10.0", "resnet50", pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 3)  # 3 classes
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

# Class labels
class_labels = ["straw_mite", "healthy", "anthracnose_leaf_spot"]

@app.post("/predict/")
async def predict(file: UploadFile):
    try:
        # Load and preprocess the image
        image = Image.open(file.file).convert("RGB")
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        img_tensor = transform(image).unsqueeze(0)

        # Perform prediction
        outputs = model(img_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[0]
        predicted_idx = torch.argmax(probabilities).item()
        predicted_label = class_labels[predicted_idx]
        confidence = probabilities[predicted_idx].item()

        return {
            "predicted_class": predicted_label,
            "confidence": confidence
        }

    except Exception as e:
        return {"error": f"Prediction error: {str(e)}"}

# Start FastAPI server using ngrok
ngrok.set_auth_token("2qi******************************************UA")      #You can make an account in ngrok and put your own auth_token here. After signing up , you can log in to check your auth_token on the left hand side dashboard
public_url = ngrok.connect(8000)
print("FastAPI is running at:", public_url)

# Start the FastAPI app (this should be non-blocking)
nest_asyncio.apply()  # Needed to allow the server to run in Colab
uvicorn.run(app, host="0.0.0.0", port=8000)




'''
ngrok is a tool that creates secure tunnels to your localhost, allowing you to expose a local server to the internet.
It is widely used for testing and debugging web applications, especially those running on your local machine.
'''

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-5-a5fe9f811b45>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a fu

FastAPI is running at: NgrokTunnel: "https://a763-34-23-88-245.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [419]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     54.86.50.139:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     54.86.50.139:0 - "POST /predict/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [419]
